In [1]:
from agentex import Agentex

client = Agentex(base_url="http://localhost:5003")

In [ ]:
AGENT_NAME = "ab030-tracing"

In [3]:
# (REQUIRED) Create a new task. For Agentic agents, you must create a task for messages to be associated with.
import uuid

rpc_response = client.agents.create_task(
    agent_name=AGENT_NAME,
    params={
        "name": f"{str(uuid.uuid4())[:8]}-task",
        "params": {}
    }
)

task = rpc_response.result
print(task)

Task(id='13e6c10d-91eb-4410-a680-6b730ca4bf7e', created_at=datetime.datetime(2025, 7, 28, 3, 56, 47, 164208, tzinfo=TzInfo(UTC)), name='454cb225-task', status='RUNNING', status_reason='Task created, forwarding to ACP server', updated_at=datetime.datetime(2025, 7, 28, 3, 56, 47, 164208, tzinfo=TzInfo(UTC)))


In [4]:
# Send an event to the agent

# The response is expected to be a list of TaskMessage objects, which is a union of the following types:
# - TextContent: A message with just text content   
# - DataContent: A message with JSON-serializable data content
# - ToolRequestContent: A message with a tool request, which contains a JSON-serializable request to call a tool
# - ToolResponseContent: A message with a tool response, which contains response object from a tool call in its content

# When processing the message/send response, if you are expecting more than TextContent, such as DataContent, ToolRequestContent, or ToolResponseContent, you can process them as well

rpc_response = client.agents.send_event(
    agent_name=AGENT_NAME,
    params={
        "content": {"type": "text", "author": "user", "content": "Hello what can you do?"},
        "task_id": task.id,
    }
)

event = rpc_response.result
print(event)

Event(id='8b547636-205b-4f5e-9356-797de5b45de8', agent_id='ef21f2e7-5bbe-440a-824b-7e89f462a781', sequence_id=244, task_id='13e6c10d-91eb-4410-a680-6b730ca4bf7e', content=TextContent(author='user', content='Hello what can you do?', attachments=None, format='plain', style='static', type='text'), created_at=datetime.datetime(2025, 7, 28, 3, 56, 47, 689616, tzinfo=TzInfo(UTC)))


In [5]:
# Subscribe to the async task messages produced by the agent
from agentex.lib.utils.dev_tools import subscribe_to_async_task_messages

task_messages = subscribe_to_async_task_messages(
    client=client,
    task=task, 
    only_after_timestamp=event.created_at, 
    print_messages=True,
    rich_print=True,
    timeout=5,
)

╭───────────────────────── USER [07/28/2025 03:56:47] ─────────────────────────╮
│ Hello what can you do?                                                       │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [07/28/2025 03:56:47] ─────────────────────────╮
│ Hello! I can assist you with a variety of tasks, including:                  │
│                                                                              │
│  1 Answering Questions: I can provide information on a wide range of topics, │
│    from science and history to technology and culture.                       │
│  2 Providing Explanations: I can explain concepts or topics in detail,       │
│    whether they are academic, technical, or general knowledge.               │
│  3 Offering Suggestions: I can give recommendations, such as book            │
│    suggestions, movie picks, or travel destinations based on your interests. │
│  4 Writing Assistance: I can help with writing tasks, including drafting     │
│    emails, essays, or creative writing prompts.                              │
│  5 Language Translation: I can translate text between several languages.     │
│  6 Problem Solving: I can assist with problem-solving in areas like math,    │
│    coding, or brainstorming ideas.                                           │
│  7 Learning Resources: I can suggest resources for further learning on a     │
│    variety of subjects.                                                      │
│                                                                              │
│ If there's something specific you want help with, feel free to ask!          │
╰──────────────────────────────────────────────────────────────────────────────╯

Streaming timed out after 5 seconds - returning collected messages
